In [ ]:
#%% Imports:
import matplotlib
matplotlib.use('MACOSX')

from math import sqrt
import os
import numpy as np
import mne
#import matplotlib #doesnt it import through mne already?
from copy import deepcopy

#Open data:
#sample_data_folder = mne.datasets.sample.data_path()
#kath_raw_file2 = "/Users/jenya/Documents/Oldenburg and university/Job Uni Rieger lab/Katharinas_Data/sub_HT05ND16/210811/mikado-1.fif"
#kath_raw_file = os.path.join('Katharinas_Data','sub_HT05ND16', '210811', 'mikado-1.fif')
kath_raw_file='/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Katharinas_Data/sub_HT05ND16/210811/mikado-1.fif'
print(kath_raw_file)
#print(kath_raw_file2)                                   
raw = mne.io.read_raw_fif(kath_raw_file)
#raw.crop(0, 60).load_data()  # just use a fraction of data for speed here

#Print info about the data:
print(raw)
print(raw.info)
raw

#crop the data to calculate faster

raw_cropped = raw.copy()
raw_cropped.crop(0, 300) #(first 5 min)

In [ ]:
#%% Find magnetometers and gradiometers:
# unit t - magenetometer. unit M_T - gradiometer. (in this set name will end with 1 for magnet, 
# with 2and3 for grad.)

mags = []
grads = []

for i, chs in enumerate(raw_cropped.info['chs']):

    if str(chs['unit']).endswith('UNIT_T)'):
        mags.append((chs['ch_name'], i))
    elif str(chs['unit']).endswith('UNIT_T_M)'):
        grads.append((chs['ch_name'], i))

In [ ]:
#epoch channels separately
events = mne.find_events(raw_cropped, stim_channel='STI101', min_duration=1.2)

picks_grad = mne.pick_types(raw_cropped.info, meg='grad', eeg=False, eog=False, stim=False)
picks_magn = mne.pick_types(raw_cropped.info, meg='mag', eeg=False, eog=False, stim=False)

epochs_mags = mne.Epochs(raw_cropped, events, picks=picks_magn, tmin=-0.2, tmax=1, preload=True, baseline = None)
epochs_grads = mne.Epochs(raw_cropped, events, picks=picks_grad, tmin=-0.2, tmax=1, preload=True, baseline = None)

#Present epochs as data frame - separately for mags and grads

df_epochs_mags = epochs_mags.to_data_frame(time_format=None, scalings=dict(mag=1))

df_epochs_grads = epochs_grads.to_data_frame(time_format=None, scalings=dict(grad=1))

In [ ]:
df_epochs_mags

In [ ]:
df_epochs_mags['MEG0111'][0]